# 1. data preprocessing 및 dataset 구축하기

### 데이터 aug함수

In [13]:
import numpy as np
import random
from scipy.ndimage import interpolation as inter

# NOTE
# Images are assumed to be uint8 0-255 valued.
# For augment function:
#   images shape: (batch_size, height, width, channels=3)
#   labels shape: (batch_size, 3)

'''def shift(image1, image2, rate_x=0.01, rate_y=0.05):
    new_img1 = np.copy(image1)
    new_img2 = np.copy(image2)
    
    shape = new_img1.shape
    if shape[0] != new_img2.shape[0] :
        print("augment ERR!! : Diff Shift Size")
    elif shape[1] != new_img2.shape[1] :
        print("augment ERR!! : Diff Shift Size")
    
    max_x = int(shape[1] * rate_x)
    max_y = int(shape[0] * rate_y)
    x = np.random.randint(low=-max_x, high=max_x)
    y = np.random.randint(low=-max_y, high=max_y)
    new_img1 = inter.shift(new_img1,shift=[y,x,0], mode='nearest')
    new_img2 = inter.shift(new_img2,shift=[y,x,0], mode='nearest')
    
    return new_img1, new_img2'''

def crop_image(image1, image2, crop_shape=None, padding=5) :
    
    oshape = np.shape(image1)
    if crop_shape is None :
        crop_shape = image1.shape

    if padding:
        oshape = (oshape[0] + 2*padding, oshape[1] + 2*padding)

    npad = ((padding, padding), (padding, padding), (0, 0))
    
    out_image1 = image1
    out_image2 = image2
    if padding:
        out_image1 = np.lib.pad(image1, pad_width=npad, mode='constant', constant_values=0)
        out_image2 = np.lib.pad(image2, pad_width=npad, mode='constant', constant_values=0)
    nh = random.randint(0, oshape[0] - crop_shape[0])
    nw = random.randint(0, oshape[1] - crop_shape[1])
    out_image1 = out_image1[nh:nh + crop_shape[0], nw:nw + crop_shape[1]]
    out_image2 = out_image2[nh:nh + crop_shape[0], nw:nw + crop_shape[1]]
    return out_image1, out_image2

def rotate_180(image1, image2):
    randnum = random.getrandbits(1) * 2
    return np.rot90(image1,randnum), np.rot90(image2,randnum)

#randomly manipulates image
#rotate, flip along axis, add blotch, shift
def augment(images1, images2, labels=None, amplify=2):
    # INPUT:
    #images shape: (batch_size, height, width, channels=3)
    #labels shape: (batch_size, 3)
    ops = [
        crop_image,
        rotate_180
    ] 

    shape1 = images1.shape
    shape2 = images2.shape
    #print("img", shape1, "lab", labels.shape)
    if shape1[0] != shape2[0] :
        print("augment ERR!! : Diff Batch Size")
    new_images1 = np.zeros(((amplify*shape1[0]), shape1[1], shape1[2], shape1[3]))
    new_images2 = np.zeros(((amplify*shape2[0]), shape2[1], shape2[2], shape2[3]))
    if labels is not None :
        new_labels = np.zeros(amplify*shape1[0])
    for i in range(shape1[0]):
        new_images1[i] = np.copy(images1[i])
        new_images2[i] = np.copy(images2[i])
        if labels is not None :
            new_labels[i] = labels[i]
        for j in range(1, amplify):
            add_r = ( j * shape1[0] )
            tmp_img1 = np.copy(new_images1[i])
            tmp_img2 = np.copy(new_images2[i])
            for each_op in ops :
                tmp_img1, tmp_img2 = each_op(tmp_img1, tmp_img2)
            new_images1[i+add_r] = tmp_img1
            new_images2[i+add_r] = tmp_img2
            if labels is not None :
                new_labels[i+add_r] = labels[i]
    if labels is not None :
            return new_images1, new_images2, new_labels.astype(np.uint8).astype(np.str_)
    else:
        return new_images1, new_images2
    
def banknoteAugment(in_image_Dic, train_size, in_label_Dic=None, batch_size=10, ample_size=10) :
    ### Start about Image Data Augmented ###
    in_train_shape = (len(in_image_Dic[typeName_list[0]]), len(in_image_Dic[typeName_list[0]][0]), 
                      len(in_image_Dic[typeName_list[0]][0][0]), len(in_image_Dic[typeName_list[0]][0][0][0]))
    if in_label_Dic is not None  :
        in_lable_shape = (len(in_label_Dic[typeName_list[0]]), )
        
    x_trainAugm = {}
    y_trainAugm = {}
    out_image_Dic = {}
    out_label_Dic = {}
    
    print("img",in_train_shape, "lab", in_lable_shape)
    
    for tmp_type in typeName_list:
        x_trainAugm[tmp_type] = np.empty((0,) + in_train_shape[1:])
        if in_label_Dic is not None :
            y_trainAugm[tmp_type] = np.empty((0,) + in_lable_shape[1:])

    n = batch_size 
    ample_size = 10
    for i in range(0, train_size, batch_size):
        i_end = min(train_size, i + batch_size)
        # call augment func
        if in_label_Dic is not None :
            x1_augm, x2_augm, y_augm = augment(images1=np.array(in_image_Dic[typeName_list[0]][i:i_end]), images2=np.array(in_image_Dic[typeName_list[1]][i:i_end]), 
                                               labels=np.array(in_label_Dic[typeName_list[0]][i:i_end]), amplify=ample_size)
        else :
            x1_augm, x2_augm = augment(images1=np.array(in_image_Dic[typeName_list[0]][i:i_end]), images2=np.array(in_image_Dic[typeName_list[1]][i:i_end]), amplify=ample_size)
            
        #sum each epohc
        x_trainAugm[typeName_list[0]] = np.append(x_trainAugm[typeName_list[0]], x1_augm, axis=0)
        x_trainAugm[typeName_list[1]] = np.append(x_trainAugm[typeName_list[1]], x2_augm, axis=0)
        if in_label_Dic is not None :
            y_trainAugm[typeName_list[0]] = np.append(y_trainAugm[typeName_list[0]], y_augm, axis=0)
            y_trainAugm[typeName_list[1]] = np.append(y_trainAugm[typeName_list[1]], y_augm, axis=0)
        """if i == 0 :
            print(x_trainAugm[typeName_list[0]].shape, x1_augm.shape)"""
        if (i + 1)%100 == 0:
            print()
            print(str(i)+' :',end=' ')
        else:
            print('.',end='')

    print()   
    #make new data list [ new = train(augm) + test(old) ]
    for tmp_type in typeName_list:
        out_image_Dic[tmp_type] = list(np.append(x_trainAugm[tmp_type], in_image_Dic[tmp_type][train_size:], axis=0))
        if in_label_Dic is not None :
            out_label_Dic[tmp_type] = list(np.append(y_trainAugm[tmp_type], in_label_Dic[tmp_type][train_size:], axis=0))
            
    if in_label_Dic is not None :
        return out_image_Dic, out_label_Dic
    return out_image_Dic

In [14]:
import numpy as np
#import cv2
# show error cases
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from keras.utils import np_utils

num_classes = 2
dataset_dir = "../dataset"
labelName_list = ["0", "1"]

channelName_list = ["BLUE", "GREEN", "IRR", "IRT", "RED"]
compChannel_Inten_Dic = {"BLUE" : 0.0722, "GREEN" : 0.7152, "IRR" : 0, "IRT" : 0, "RED" : 0.2126}

headingName_list = ["A", "B", "C", "D"]
flagToType_Dic = {"A" : [0, 1], "B" : [0, 1], "C" : [1, 0], "D" : [1, 0]}

typeName_list = ["FRONT", "BACK"]

def shuffle_data(data, label):
    idx = np.arange(len(data))
    np.random.shuffle(idx)
    
    shuffled_data = np.array(data)[np.array(idx)]
    shuffled_label = np.array(label)[np.array(idx)]

    return shuffled_data, shuffled_label, idx

def shuffle_data_in_defined(data, label, idx):
    shuffled_data = np.array(data)[np.array(idx)]
    shuffled_label = np.array(label)[np.array(idx)]
    
    return shuffled_data, shuffled_label, idx
    
def load_data(parent_dir, ext):
    
    image_Front_list = []
    image_Back_list = []
    label_Front_list = []
    label_Back_list = []
    fileFrontPath_list = []
    fileBackPath_list = []
    
    image_Dic = {'FRONT' : image_Front_list, 'BACK' : image_Back_list}
    label_Dic = {'FRONT' : label_Front_list, 'BACK' : label_Back_list}
    filePath_Dic = {'FRONT' : fileFrontPath_list, 'BACK' : fileBackPath_list}

    
    labelNames = os.listdir(parent_dir)
    for labelName in labelNames: # 하위 디렉토리 탐색      (label 폴더 : 0, 1)
        if not(any(labelName in tmpstr for tmpstr in labelName_list)):
            print("head ERR!!!")
            continue  # 해당 폴더가 투입면 폴더가 아닌 경우 다시 검색
        labelDirPath = os.path.join(parent_dir, labelName)
        print(labelName)
        if os.path.isdir(labelDirPath): # 디렉토리인 경우
            headingNames = os.listdir(labelDirPath)
            for headingName in headingNames: # 하위 디렉토리 탐색      (heading 폴더 : A, B, C, D)
                if not(any(headingName in tmpstr for tmpstr in headingName_list)):
                    print("head ERR!!!")
                    continue  # 해당 폴더가 투입면 폴더가 아닌 경우 다시 검색
                print('_' + headingName)
                headingDirPath = os.path.join(labelDirPath, headingName)

                if os.path.isdir(headingDirPath): # 디렉토리인 경우
                    channelNames = os.listdir(headingDirPath)
                    
                    #로드 리스트 생성
                    loadingImg_list = []
                    loadinglable_list = []
                    loadingName_list = []
                    
                    #합성을 위한 order dic
                    orderChannel_Dic = {}
                    
                    for cnt, channelName in enumerate(channelNames):  # 하위 디렉토리 탐색      (각 Channel 접근 : R, G, B, IRR, IRT)
                        if not(any(channelName in tmpstr for tmpstr in channelName_list)):
                            print("channel ERR!!!")
                            continue  # 해당 폴더가 채널폴더가 아닌 경우 다시 검색
                        print('__' + channelName)
                        fileDirPath = os.path.join(headingDirPath, channelName)
                        
                        #순서 저장
                        orderChannel_Dic[channelName] = cnt
                        
                        if os.path.isdir(fileDirPath): # 디렉토리인 경우
                            fileNames = os.listdir(fileDirPath)
                            for fileName in fileNames: #해당폴더 전체 이미지 탐색
                                # 순서대로 새로운 채널에 입력
                                filePath = os.path.join(fileDirPath, fileName)

                                _ext = os.path.splitext(filePath)[-1] # 파일 확장자 확인
                                if _ext == ext:
                                    if len(loadingImg_list)% 130 == 0:
                                        print()
                                    print('.', end = '')
                                    image = mpimg.imread(filePath)
                                    if image is None:
                                        continue
                                
                                #이미지 크기 정규화
                                image = image.astype(np.float32) / 255.
                                #image = image.reshape(65, 308)
                                
                                #있는지 확인
                                if any(fileName in tmpstr for tmpstr in loadingName_list): #파일이 있는지 없는지 확인
                                    #있으면 dstack
                                    tmpidx = loadingName_list.index(fileName)
                                    loadingImg_list[tmpidx] = np.dstack((loadingImg_list[tmpidx], image))
                                    #shapetmp = loadingImg_list[tmpidx].shape
                                    
                                    if channelName == "IRT":
                                        nameListLen = int(len(loadingName_list)/2 -1)
                                        #print(type(nameListLen))
                                        #print(loadingName_list)
                                        for chatchName in loadingName_list[nameListLen:] :
                                            if fileName[5:] in chatchName:
                                                findedName = chatchName
                                        tmp = loadingName_list.index(findedName)
                                        loadingImg_list[tmp] = np.dstack((loadingImg_list[tmp], image))
                                    
                                    #if tmp == 0:
                                        #print("!" + fileName + "\t" + str(tmp) + "\t" + str(len(loadingImg_list)) + "\t" +  str(image.shape) + "\t" +  str(shapetmp) + "\t" +  str(loadingImg_list[tmp].shape))
                                    #print("!" + fileName + "\t" + str(len(loadingImg_list)) + "\t" +  str(image.shape) + "\t" +  str(shapetmp))
                                else:
                                    #없으면 append
                                    loadingImg_list.append(image)
                                    #label & name 추가
                                    #loadinglable_list.append(str(len(loadingName_list))+"\t"+labelName + headingName + fileName)
                                    loadinglable_list.append(labelName)
                                    loadingName_list.append(fileName)
                                    #print("?" + fileName)
                            print()

                    #RGB를 하나로 압축
                    compImg_list = []
                    for loadedImg in loadingImg_list :
                        tmpFlag  = 0
                        rgbCnt = 0
                            
                        tmpComp_list = []
                        for cnt, channelName in enumerate(channelNames) :
                            chIdx = orderChannel_Dic[channelName]
                            chComp = compChannel_Inten_Dic[channelName]

                            if chComp == 0 :#초기화 && IRR, IRT 일때
                                if tmpFlag == 0 :
                                    tmpImg = loadedImg[:,:,chIdx]
                                    tmpFlag = 1
                                else :
                                    tmpImg = np.dstack((tmpImg, loadedImg[:,:,chIdx]))
                            else :
                                if rgbCnt == 0 :
                                    rgbImg = loadedImg[:,:,chIdx]
                                else :
                                    rgbImg = np.dstack((rgbImg, loadedImg[:,:,chIdx]))
                                tmpComp_list.append(chComp)
                                rgbCnt += 1
                        tmpImg = np.dstack((tmpImg, np.dot(rgbImg, tmpComp_list)))
                        compImg_list.append(tmpImg)
                    #####
                    
                    loadingImg_list = compImg_list
                    #리스트에 로드된 애들 입력
                    flag = flagToType_Dic[headingName]
                    typeSize = int(len(typeName_list))
                    tmpDataSize = int(len(loadingImg_list)/typeSize)

                    for cnt, tmp in enumerate(flag):
                        tmpstr = typeName_list[tmp]
                        if labelName == labelName_list[0]:
                            image_Dic[tmpstr] = image_Dic[tmpstr] + loadingImg_list[:tmpDataSize]
                            label_Dic[tmpstr] = label_Dic[tmpstr] + loadinglable_list[:tmpDataSize]
                        else :
                            image_Dic[tmpstr] = image_Dic[tmpstr] + loadingImg_list[:tmpDataSize]
                            label_Dic[tmpstr] = label_Dic[tmpstr] + loadinglable_list[:tmpDataSize]
                         
                        target_path = os.path.join(labelDirPath, headingName)
                        target_path = os.path.join(target_path, channelName_list[-1]) # RED 이미지 경로를 저장
                        for i in range(int(len(loadingName_list)/typeSize)):
                            filePath_Dic[tmpstr].append(os.path.join(target_path, loadingName_list[i + (cnt * int(len(loadingName_list)/typeSize))]))
                            #print(tmpstr + "\t" + filePath_Dic[tmpstr][i + int(len(image_Dic[tmpstr]) - tmpDataSize)])
                            #print(tmpstr + "\t" + str(i + (cnt * int(len(loadingName_list)/typeSize))) + "\t" + str(len(filePath_Dic[tmpstr])))
                        #print("!" + tmpstr + "\t" + str(len(image_Dic[tmpstr])) + "\t" + str(len(loadingImg_list)))
                    #print(len(image_Dic[headingName]))
    return image_Dic, label_Dic, filePath_Dic


# data 불러오기
image_Dic, label_Dic, filePath_Dic = load_data(dataset_dir, '.bmp')

# type 종류 마다 개수 확인
data_len = len(label_Dic[typeName_list[0]])
for tmp_type in typeName_list:
    if data_len != len(label_Dic[tmp_type]):
        print("DataSize ERR!!!")
        exit()

# dataset 분할
train_size = int(0.7 * data_len)
valid_size = int(0.15 * data_len)
test_size = data_len - train_size - valid_size
print("data Len : " + str(data_len))
print("(train + validation + test = total)")
print(train_size, '+', valid_size, '+', test_size, '=', train_size+valid_size+test_size)

# data shuffling
shuffled_image_Dic = {}
shuffled_label_Dic = {}
shuffled_idx_Dic  = {}

for tmp_type in typeName_list:
    if tmp_type == typeName_list[0] :
        shuffled_image_Dic[tmp_type], shuffled_label_Dic[tmp_type], shuffled_idx_Dic[tmp_type] = shuffle_data(image_Dic[tmp_type], label_Dic[tmp_type])
    else :
        shuffled_image_Dic[tmp_type], shuffled_label_Dic[tmp_type], shuffled_idx_Dic[tmp_type] = shuffle_data_in_defined(image_Dic[tmp_type], label_Dic[tmp_type], shuffled_idx_Dic[typeName_list[0]])

# data augment
print("augment!")
ample_size = 10
image_Dic, label_Dic = banknoteAugment(in_image_Dic=shuffled_image_Dic, train_size=train_size, in_label_Dic=shuffled_label_Dic, batch_size=10, ample_size=ample_size)

data_len = len(label_Dic[typeName_list[0]])
train_size = train_size * ample_size

print("data Len : " + str(data_len))
print("(train + validation + test = total)")
print(train_size, '+', valid_size, '+', test_size, '=', train_size+valid_size+test_size)

# data re shuffling
for tmp_type in typeName_list:
    if tmp_type == typeName_list[0] :
        shuffled_image_Dic[tmp_type], shuffled_label_Dic[tmp_type], shuffled_idx_Dic[tmp_type] = shuffle_data(image_Dic[tmp_type], label_Dic[tmp_type])
    else :
        shuffled_image_Dic[tmp_type], shuffled_label_Dic[tmp_type], shuffled_idx_Dic[tmp_type] = shuffle_data_in_defined(image_Dic[tmp_type], label_Dic[tmp_type], shuffled_idx_Dic[typeName_list[0]])

x_train = {}
y_train = {}
x_valid = {}
y_valid = {}
x_test = {}
y_test = {}
shuffle_indices = {} # 디버그용

# dataset 구축 // one-hot 전
for tmp_type in typeName_list:
    y_train[tmp_type + str(1)] = shuffled_label_Dic[tmp_type][:train_size]
    y_valid[tmp_type + str(1)] = shuffled_label_Dic[tmp_type][train_size:data_len-test_size]
    y_test[tmp_type + str(1)] = shuffled_label_Dic[tmp_type][data_len-test_size:]
    print("Y test  : " + str(y_test[tmp_type + str(1)].shape))
    
# one-hot vector로 변환
for tmp_type in typeName_list:
    shuffled_label_Dic[tmp_type] = np_utils.to_categorical(shuffled_label_Dic[tmp_type], num_classes)

    
# dataset 구축  // one-hot 후
for tmp_type in typeName_list:
    x_train[tmp_type] = shuffled_image_Dic[tmp_type][:train_size]
    y_train[tmp_type + str(0)] = shuffled_label_Dic[tmp_type][:train_size]
    x_valid[tmp_type] = shuffled_image_Dic[tmp_type][train_size:data_len-test_size]
    y_valid[tmp_type + str(0)] = shuffled_label_Dic[tmp_type][train_size:data_len-test_size]
    x_test[tmp_type] = shuffled_image_Dic[tmp_type][data_len-test_size:]
    y_test[tmp_type + str(0)] = shuffled_label_Dic[tmp_type][data_len-test_size:]
    shuffle_indices[tmp_type] = shuffled_idx_Dic[tmp_type][data_len-test_size:] # 디버그용
    print(tmp_type)
    print("X train : " + str(x_train[tmp_type].shape))
    print("Y train : " + str(y_train[tmp_type + str(0)].shape))
    print("X test  : " + str(x_test[tmp_type].shape))
    print("Y test  : " + str(y_test[tmp_type + str(0)].shape))


0
_A
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
_B
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
_C
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
_D
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
1
_A
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
_B
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
_C
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
_D
__BLUE

..........
__GREEN
..........
__IRR
..........
__IRT
.....
__RED
..........
data Len : 40
(train + validation + test = total)
28 + 6 + 6 = 40
augment!
img (40, 65, 300, 3) lab (40,)
img (10, 65, 300, 3) lab (10,)
.img (10, 65, 300, 3) lab (10,)
.img (8, 65, 300, 3) lab (8,)
.
data Len : 292
(train + validation + test = total)
280 + 6 + 6 = 292
Y test  : (6,)
Y test  : (6,)


In [23]:
typeNum = 2
verNum = 3
projectName_list = ["2_Type1_ver2", "2_Type1_ver3", "2_Type1_ver4", "2_Type2_ver2", "2_Type2_ver3", "2_Type2_ver4"]

In [24]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation
from keras.models import Sequential, save_model

for cnt, projectName_str in enumerate(projectName_list):
    print()
    print(str(cnt) + " : " + projectName_str)
    print()
    for tmpType in typeName_list:
        path_each_str = "/" + tmpType + "/"
        ytmpType = tmpType + str((cnt%verNum)//(verNum -1))
        model = Sequential()

        #각기 다른 방식으로 train 진행
        if cnt < len(typeName_list)/typeNum :
            model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid', input_shape= x_train[tmpType].shape[1:]))
            model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            #model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Dropout(0.25))

            model.add(Flatten())
            model.add(Dense(50, activation='relu'))

        else :
            model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid', input_shape= x_train[tmpType].shape[1:]))
            model.add(Conv2D(filters= 16, kernel_size = (3,3), activation='relu', padding='valid'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 64, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Conv2D(filters= 64, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 64, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Conv2D(filters= 64, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(MaxPooling2D(pool_size= (2, 2)))

            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))
            model.add(Conv2D(filters= 32, kernel_size = (3,3), activation='relu', padding='same'))

            model.add(Flatten())
            model.add(Dense(50, activation='relu'))
            
        if (cnt%verNum) == 0 :
            model.add(Dense(2))
            model.add(Activation('softmax'))

            # configure the model
            model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
        elif (cnt%verNum) == 1:
            model.add(Dense(2))
            model.add(Activation('sigmoid'))

            # configure the model
            model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
        elif (cnt%verNum) == 2:
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            # configure the model
            model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
        else :
            print("verNum ERR!!!")
            continue

        # prints a summary representation of the model
        model.summary()

        # set tensorboard
        from keras.callbacks import TensorBoard
        tbCallback = TensorBoard(log_dir='./tensorboard/' + path_each_str + projectName_str)
        tbCallback.set_model(model)

        # save the model
        if not os.path.exists('./logs' + path_each_str):
            os.makedirs('./logs' + path_each_str)
        save_model(model, './logs' + path_each_str + 'my_model_' + projectName_str + '.hdf5')

        # make parameters checkpoint
        from keras.callbacks import ModelCheckpoint
        checkpoint = ModelCheckpoint(filepath="./logs/" + path_each_str + "weights.best_" + projectName_str + ".hdf5", verbose=1, 
                                       monitor='loss', save_best_only=True, mode='auto')

        # # set early stopping
        from keras.callbacks import EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)

        
        
        
        # train the model
        model.fit(x_train[tmpType], y_train[ytmpType],
                        epochs=50,
                        batch_size=256,
                        shuffle=False, # previously shuffled
                        validation_data=(x_valid[tmpType], y_valid[ytmpType]),
                        callbacks = [tbCallback, checkpoint, early_stopping])

        '''
        learning is done at this point
        '''
        score = model.evaluate(x_test[tmpType], y_test[ytmpType], verbose=0)

        print('Test loss: ', score[0])
        print('Test accuracy: ', score[1])
print("end")


0 : 2_Type1_ver2

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 298, 16)       448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 296, 16)       2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 148, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 148, 32)       4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 74, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 74, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 37, 32)         0 

## Test Set

In [25]:
from keras.models import load_model

nameTokenWord = "-V-"
splitToken = "_"

for cnt, projectName_str in enumerate(projectName_list):
    print()
    print(str(cnt) + " : " + projectName_str)
    
    for tmpType in typeName_list:
        print()
        print(tmpType)
        print()
        path_each_str = "/" + tmpType + "/"
        ytmpType = tmpType + str((cnt%verNum)//(verNum -1))
        # restore trained model
        loaded_model = load_model('./logs' + path_each_str + 'my_model_' + projectName_str + '.hdf5')
        loaded_model.load_weights("./logs/" + path_each_str + "weights.best_" + projectName_str + ".hdf5")

        size_cnt = 0
        FA_cnt = 0
        FR_cnt = 0
        truesize = 0
        falsesize = 0
        
        if (cnt%verNum) != (verNum -1):    
            # predict test data
            predict = loaded_model.predict(x_test[tmpType])
            
            for i in range(test_size):
                if np.argmax(y_test[ytmpType][i]) == 0 :
                    truesize += 1
                else :
                    falsesize += 1

                if not (np.argmax(predict[i]) == np.argmax(y_test[ytmpType][i])):
                    tmpidx = filePath_Dic[tmpType][shuffle_indices[tmpType][i]].find(nameTokenWord)
                    outStr = filePath_Dic[tmpType][shuffle_indices[tmpType][i]][tmpidx+3:-4].split(splitToken)
                    print(''.join(outStr))
                    print(str(np.argmax(y_test[ytmpType][i])) + '\t' + str(np.argmax(predict[i])))
                    size_cnt += 1
                    if np.argmax(y_test[ytmpType][i]) == 1:
                        FA_cnt += 1
                    else :
                        FR_cnt += 1

            truesize = 0
            falsesize = 0
            for i in range(test_size):
                if np.argmax(y_test[ytmpType][i]) == 0 :
                    truesize += 1
                else :
                    falsesize += 1
                print(str(predict[i][0]) + '\t' + str(predict[i][1]) + '\t' + str(np.argmax(predict[i])) + '\t' + str(np.argmax(y_test[ytmpType][i])))
            print('incorrect:', str(size_cnt) + '/' + str(test_size), 'False Accept:', str(FA_cnt) + '/' + str(falsesize), 'False Reject:', str(FR_cnt) + '/' + str(truesize), 'accuracy:', (test_size-size_cnt)/test_size)
            print('incorrect:', str(round((size_cnt/test_size)*100, 2)) + '%', 'False Accept:', str(round((FA_cnt/falsesize)*100, 2)) + '%', 'False Reject: ', str(round((FR_cnt/truesize)*100, 2)) + '%')
        else :
             # predict test data
            predict_data = loaded_model.predict(x_test[tmpType], verbose= 1)
            predict = loaded_model.predict_classes(x_test[tmpType], verbose= 1)
            
            for i in range(test_size):
                if int(y_test[ytmpType][i]) == 0 :
                    truesize += 1
                else :
                    falsesize += 1
                    
                if not (predict[i][0] == int(y_test[ytmpType][i])):
                    tmpidx = filePath_Dic[tmpType][shuffle_indices[tmpType][i]].find(nameTokenWord)
                    outStr = filePath_Dic[tmpType][shuffle_indices[tmpType][i]][tmpidx+3:-4].split(splitToken)
                    print(''.join(outStr))
                    print(str(y_test[ytmpType][i]) + '\t' + str(predict[i][0]))
                    size_cnt += 1
                    if int(y_test[ytmpType][i]) == 1:
                        FA_cnt += 1
                    else :
                        FR_cnt += 1
                        
            truesize = 0
            falsesize = 0
            for i in range(test_size):
                if int(y_test[ytmpType][i]) == 0 :
                    truesize += 1
                else :
                    falsesize += 1
                print(str(predict_data[i][0]) + '\t' + str(predict[i][0]) + '\t' + str(y_test[ytmpType][i]))
            print('incorrect:', str(size_cnt) + '/' + str(test_size), 'False Accept:', str(FA_cnt) + '/' + str(falsesize), 'False Reject:', str(FR_cnt) + '/' + str(truesize), 'accuracy:', (test_size-size_cnt)/test_size)
            print('incorrect:', str(round((size_cnt/test_size)*100, 2)) + '%', 'False Accept:', str(round((FA_cnt/falsesize)*100, 2)) + '%', 'False Reject: ', str(round((FR_cnt/truesize)*100, 2)) + '%')
print("end")


0 : 2_Type1_ver2

FRONT

000000012055100000001
1	0
000000012042300000001
1	0
000000032050200000003
1	0
0000000118085000000001
1	0
000000022050100000002
1	0
000000052084300000005
1	0
0000000116181900000001
0	1
000000042083600000004
1	0
0000000516151300000005
0	1
000000012072300000001
1	0
0000000218090800000002
1	0
0000000518045700000005
1	0
000000030504800000003
0	1
000000012063600000001
1	0
000000032033800000003
1	0
000000020412300000002
0	1
0000000518090300000005
1	0
000000032091700000003
1	0
000000042071800000004
1	0
0000000518051600000005
1	0
0000000218032400000002
1	0
0000000518073900000005
1	0
0000000318065600000003
1	0
000000052034500000005
1	0
0.999991	8.69596e-06	0	0
0.99677	0.00323031	0	0
0.995432	0.00456807	0	0
0.999888	0.00011175	0	0
0.999993	6.99656e-06	0	0
0.999958	4.21947e-05	0	0
0.999986	1.36807e-05	0	0
0.948732	0.0512679	0	0
0.99869	0.00130961	0	0
0.999943	5.70728e-05	0	0
0.999998	1.6746e-06	0	0
0.995031	0.00496867	0	0
0.999993	6.86967e-06	0	0
0.960249	0.0397505	0	0
0.